## SVM

In [1]:
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from termcolor import colored
from collections import Counter
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from imblearn.over_sampling import SMOTE

In [2]:
data=pd.read_csv("C:/Users/Administrator/2023_Data_Mining_Coffee_Quality_Dataset/cleaned_data.csv")

In [3]:
data.drop("Unnamed: 0",axis=1,inplace=True)

In [4]:
Y=data["Total.Cup.Points"]
X=data.drop("Total.Cup.Points",axis=1)
feature_names=X.columns
X.head()

,Species,Country.of.Origin,Region,Harvest.Year,Variety,Processing.Method,Category.One.Defects,Quakers,Color,Category.Two.Defects,altitude_mean_meters
0,0,8,115,5,5,4,0,0.0,2,0,2075.0
1,0,8,115,5,15,4,0,0.0,2,1,2075.0
2,0,9,122,3,2,4,0,0.0,2,0,1700.0
3,0,8,226,5,5,0,0,0.0,2,2,2000.0
4,0,8,115,5,15,4,0,0.0,2,2,2075.0


In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.30, stratify=Y, random_state=42)
print(X_train.shape, X_test.shape)

(928, 11) (398, 11)


In [6]:
def resample(sampler, X, Y):
    X_resampled, y_resampled = sampler.fit_resample(X, Y)
    print(colored("Number of instances before resampling with {} : {}. ".format(type(sampler).__name__,\
                                                                           Counter(Y).items()), "green"))
    print(colored("Number of instances after  resampling with {} : {}. ".format(type(sampler).__name__,\
                                                                           Counter(y_resampled).items()), "blue"))
    return X_resampled, y_resampled
    

In [7]:
def report_imbalanced(model, x, y, text = "training"):
    y_pred = model.predict(x)
    
    print(colored("Classification report for model {} on {} data".format(type(model).__name__, text), "green"))
    print("---------------------------------------------------------------------------------")
    print(classification_report_imbalanced(y, y_pred, zero_division=True))
    print("---------------------------------------------------------------------------------")
    
    print(colored("Confusion matrix for model {} on {} data ".format(type(model).__name__, text), "green"))
    print("---------------------------------------------------------------------------------")
    print(pd.DataFrame(confusion_matrix(y, y_pred),columns=[4,5,6,7,8,9],index=[4,5,6,7,8,9]))
    print("---------------------------------------------------------------------------------")

In [8]:
#Balansiranje klasa koristeci SMOTE
smote = SMOTE(k_neighbors=10, random_state=42)
X_resampled, y_resampled = resample(smote, X_train, Y_train)

NameError: name 'colored' is not defined

In [ ]:
params = [
            {
                'kernel': ['linear'],
                'C': [0.01, 0.1, 1, 10],
            },
            {
                'kernel': ['rbf'],
                'C': [0.01, 0.1, 1, 10],
                'gamma': [0.01, 0.1, 1, 10],
            },
         ]

In [ ]:
gs = GridSearchCV(SVC(), params, scoring='accuracy', cv=5, verbose=5)

In [ ]:
gs.fit(X_train, Y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV 1/5] END .............C=0.01, kernel=linear;, score=0.602 total time= 3.4min
[CV 2/5] END .............C=0.01, kernel=linear;, score=0.597 total time= 2.2min
[CV 3/5] END .............C=0.01, kernel=linear;, score=0.602 total time= 1.7min
[CV 4/5] END .............C=0.01, kernel=linear;, score=0.600 total time= 2.9min
[CV 5/5] END .............C=0.01, kernel=linear;, score=0.605 total time= 3.1min
[CV 1/5] END ..............C=0.1, kernel=linear;, score=0.608 total time= 4.1min
[CV 2/5] END ..............C=0.1, kernel=linear;, score=0.597 total time= 4.7min
[CV 3/5] END ..............C=0.1, kernel=linear;, score=0.597 total time= 4.2min


In [ ]:
gs.best_params_

In [ ]:
gs.best_score_

In [ ]:
report_imbalanced(gs.best_estimator_,X_resampled,y_resampled)

In [ ]:
report_imbalanced(gs.best_estimator_,X_test,Y_test)